# 도구 (Tools)
도구(Tool)는 에이전트, 체인 또는 LLM이 외부 세계와 상호작용하기 위한 인터페이스입니다.

LangChain 에서 기본 제공하는 도구를 사용하여 쉽게 도구를 활용할 수 있으며, 사용자 정의 도구(Custom Tool) 를 쉽게 구축하는 것도 가능합니다.

LangChain 통합된 도구 리스트: https://python.langchain.com/v0.1/docs/integrations/tools/

- 실습 tool 예제목록
1. Python REPL
2. 검색 tool
3. 이미지 생성 tool
4. 구글 뉴스기사 검색 도구
5. 사용자 정의 도구(Custom Tool)

### 빌트인 도구(built-in tools)
랭체인에서 제공하는 사전에 정의된 도구(tool) 와 툴킷(toolkit) 을 사용할 수 있습니다.

tool 은 단일 도구를 의미하며, toolkit 은 여러 도구를 묶어서 하나의 도구로 사용할 수 있습니다.

관련 도구는 아래의 링크에서 참고하실 수 있습니다.

**참고**

LangChain Tools/Toolkits: https://python.langchain.com/docs/integrations/tools/


### Python REPL 도구
- 이 도구는 Python 코드를 REPL(Read-Eval-Print Loop) 환경에서 실행하기 위한 두 가지 주요 클래스를 제공합니다 - PythonREPLTool

**설명**

- Python 셸 환경을 제공합니다.
- 유효한 Python 명령어를 입력으로 받아 실행합니다.
- 결과를 보려면 print(...) 함수를 사용해야 합니다.

**주요 특징**

- sanitize_input: 입력을 정제하는 옵션 (기본값: True)
- python_repl: PythonREPL 인스턴스 (기본값: 전역 범위에서 실행)

**사용 방법**

- PythonREPLTool 인스턴스 생성
- run 또는 arun, invoke 메서드를 사용하여 Python 코드 실행

**입력 정제**

- 입력 문자열에서 불필요한 공백, 백틱, 'python' 키워드 등을 제거합니다.

In [1]:
from dotenv import load_dotenv
load_dotenv()

True

In [2]:
from langchain_teddynote import logging
logging.langsmith("CH15-Tools")

LangSmith 추적을 시작합니다.
[프로젝트명]
CH15-Tools


In [3]:
import warnings
warnings.filterwarnings("ignore")

In [4]:
# PythonREPLTool : 파이썬 코드를 실행하게 하는 코드
from langchain_experimental.tools import PythonREPLTool

python_tool = PythonREPLTool()

print(python_tool.invoke("print(100+200)"))

Python REPL can execute arbitrary code. Use with caution.


300



아래는 LLM 에게 파이썬 코드를 작성하도록 요청하고 결과를 반환하는 예제
### 흐름 정리

1. LLM 모델에게 특정 작업을 수행하는 Python 코드를 작성하도록 요청합니다.
2. 작성된 코드를 실행하여 결과를 얻습니다.
3. 결과를 출력합니다.

In [11]:
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnableLambda

def print_and_execute(code, debug=True):
    if debug:
        print("Code")
        print(code)

    return python_tool.invoke(code)

In [12]:
prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are Raymond Hetting, an expert python programmer, well versed in meta-programming and elegant, concise and short but well documented code. You follow the PEP8 style guide. "
            "Return only the code, no intro, no explanation, no chatty, no markdown, no code block, no nothing. Just the code.",
        ),
        ("human",
         "{input}"
         )
    ]
)

llm = ChatOpenAI(model="gpt-4o-mini", temperature=0)

chain = prompt | llm | StrOutputParser() | RunnableLambda(print_and_execute)

In [13]:
chain.invoke("로또 번호 생성기를 출력하느 코드를 작성하세요")

Code
import random

def generate_lotto_numbers():
    """Generate a set of 6 unique lotto numbers from 1 to 45."""
    return sorted(random.sample(range(1, 46), 6))

if __name__ == "__main__":
    print(generate_lotto_numbers())


''

In [14]:
import random

def generate_lotto_numbers():
    """Generate a set of 6 unique lotto numbers from 1 to 45."""
    return sorted(random.sample(range(1, 46), 6))

if __name__ == "__main__":
    print(generate_lotto_numbers())

[11, 12, 17, 20, 39, 43]


# TavilySearchResults
- Tavily 검색 API를 쿼리하고 JSON 형식의 결과를 반환합니다.
- 포괄적이고 정확하며 신뢰할 수 있는 결과에 최적화된 검색 엔진입니다.
- 현재 이벤트에 대한 질문에 답변할 때 유용합니다.

**주요 매개변수**

- max_results (int): 반환할 최대 검색 결과 수 (기본값: 5)
- search_depth (str): 검색 깊이 ("basic" 또는 "advanced") ??
- include_domains (List[str]): 검색 결과에 포함할 도메인 목록
- exclude_domains (List[str]): 검색 결과에서 제외할 도메인 목록
- include_answer (bool): 원본 쿼리에 대한 짧은 답변 포함 여부
- include_raw_content (bool): 각 사이트의 정제된 HTML 콘텐츠 포함 여부
- include_images (bool): 쿼리 관련 이미지 목록 포함 여부

**반환 값**

- 검색 결과를 포함하는 JSON 형식의 문자열(url, content) 혹은 아래의 주석을 해제하고 발급받은 API 키를 입력합니다.

In [16]:
from langchain_community.tools.tavily_search import TavilySearchResults

# 도구생성
# tool = TavilySearchResults()

tool = TavilySearchResults(
    max_results=6,
    include_answer=True,
    include_raw_content=True,
    # include_images=True, 
    # search_depth="advanced", # or "basic"
    include_domains=["github.io", "wikidocs.net"],
    # exclude_domains = []   
)
# 도구 실행
tool.invoke({"query": "LangChain Tools 에 대해서 알려주세요"})

[{'url': 'https://wikidocs.net/262582',
  'content': '2 days ago · 도구(Tool)는 에이전트, 체인 또는 LLM이 외부 세계와 상호작용하기 위한 인터페이스입니다. LangChain 에서 기본 제공하는 도구를 사용하여 쉽게 도구를\xa0...'},
 {'url': 'https://teddylee777.github.io/langchain/langchain-agent/',
  'content': '태그:\nAgent,\nAPI KEY,\nFAISS,\nLangChain,\nLangSmith,\nmemory,\nOpenAI,\nPython,\nRetriever,\nTavily Search,\ntools,\n검색도구,\n랭체인,\n에이전트\n카테고리:\nlangchain\n업데이트: 2024년 02월 09일\n참고\nLangChain RAG 파헤치기: 다음의 추적 링크에서 자세한 단계별 수행 결과를 확인할 수 있습니다\nLangSmith 추적\n다음의 추적 링크에서 자세한 단계별 수행 결과를 확인할 수 있습니다\nLangSmith 추적\n④ 메모리 추가하기\n앞서 언급했듯이, 이 에이전트는 상태가 없습니다. LangChain 한국어 튜토리얼\n바로가기 👀\n[LangChain] 에이전트(Agent)와 도구(tools)를 활용한 지능형 검색 시스템 구축 가이드\n2024년 02월 09일\n41 분 소요\n이 글에서는 LangChain 의 Agent 프레임워크를 활용하여 복잡한 검색과 📍 전체 템플릿 코드\n다음의 추적 링크에서 자세한 단계별 수행 결과를 확인할 수 있습니다\nLangSmith 추적\n마무리입니다!\n 문서 기반 QA 시스템 설계 방법 - 심화편\n2024년 02월 06일\n22 분 소요\nLangChain의 RAG 시스템을 통해 문서(PDF, txt, 웹페이지 등)에 대한 질문-답변을 찾는 과정을 정리하였습니다.\n'},
 {'url': 'https://13akstjq.github.io/TIL/post

# Image 생성 도구 (DALL-E)
- DallEAPIWrapper 클래스: OpenAI의 DALL-E 이미지 생성기를 위한 래퍼(wrapper)입니다.
이 도구를 사용하면 DALL-E API를 쉽게 통합하여 텍스트 기반 이미지 생성 기능을 구현할 수 있습니다. 다양한 설정 옵션을 통해 유연하고 강력한 이미지 생성 도구로 활용할 수 있습니다.

**주요 속성**

- model: 사용할 DALL-E 모델 이름 (기본값: "dall-e-2", "dall-e-3")

- n: 생성할 이미지 수 (기본값: 1)

- size: 생성할 이미지 크기

    - "dall-e-2": "1024x1024", "512x512", "256x256"
    - "dall-e-3": "1024x1024", "1792x1024", "1024x1792"
- style: 생성될 이미지의 스타일 (기본값: "natural", "vivid")

- quality: 생성될 이미지의 품질 (기본값: "standard", "hd")

- max_retries: 생성 시 최대 재시도 횟수

**주요 기능**

- DALL-E API를 사용하여 텍스트 설명에 기반한 이미지 생성
**흐름 정리**

다음은 DALL-E Image Generator 를 사용하여 이미지를 생성하는 예제입니다.

이번에는 DallEAPIWrapper 를 사용하여 이미지를 생성해 보겠습니다.

이때 입력 프롬프트는 LLM 모델에게 이미지를 생성하는 프롬프트를 작성하도록 요청합니다.

In [20]:
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import PromptTemplate
from langchain_openai import ChatOpenAI

# ChatOpenAI 모델 초기화
llm = ChatOpenAI(model="gpt-4o-mini", temperature=0.9, max_tokens=1000)

# DALL-E 이미지 생성을 위한 프롬프트 템플릿 정의
prompt = PromptTemplate.from_template(
    "Generate a detailed IMAGE GENERATION prompt for DALL-E based on the following description. "
    "Return only the prompt, no intro, no explanation, no chatty, no markdown, no code block, no nothing. Just the prompt"
    "Output should be less than 1000 characters. Write in English only."
    "Image Description: \n{image_desc}",
)

# 프롬프트, LLM, 출력 파서를 연결하는 체인 생성
chain = prompt | llm | StrOutputParser()

# 체인 실행
image_prompt = chain.invoke(
    #{"image_desc": "스마트폰을 바라보는 사람들을 풍자한 neo-classicism painting"}
    {"image_desc": "한글날을 바라보며 흐뭇해하는 세종대왕 그림"}
)

# 이미지 프롬프트 출력
print(image_prompt)

A portrait of King Sejong the Great, joyfully gazing at a scene celebrating Hangul Day. The background features colorful banners and traditional Korean decorations symbolizing the importance of the Korean language. Sejong, dressed in traditional royal attire, wears a serene and content expression, with a subtle smile playing on his lips. In one hand, he holds a stylized version of the Hangul alphabet, while the other hand gestures towards a group of diverse people, young and old, joyfully reading and writing in Hangul. The setting is a harmonious blend of historical and contemporary elements, showcasing both the beauty of traditional Korean culture and the vibrant spirit of modern society. Soft, warm lighting enhances the feeling of celebration and unity, making the scene both nostalgic and uplifting.


In [21]:
# DALL-E API 래퍼 가져오기
from langchain_community.utilities.dalle_image_generator import DallEAPIWrapper
from IPython.display import Image

# DALL-E API 래퍼 초기화
# model: 사용할 DALL-E 모델 버전
# size: 생성할 이미지 크기
# quality: 이미지 품질
# n: 생성할 이미지 수
dalle = DallEAPIWrapper(model="dall-e-3", size="1024x1024", quality="standard", n=1)

# 질문
query = "한글날을 바라보며 흐뭇해하는 세종대왕 그림"

# 이미지 생성 및 URL 받기
# chain.invoke()를 사용하여 이미지 설명을 DALL-E 프롬프트로 변환
# dalle.run()을 사용하여 실제 이미지 생성
image_url = dalle.run(chain.invoke({"image_desc": query}))

# 생성된 이미지를 표시합니다.
Image(url=image_url, width=500)

# 사용자 정의 도구(Custom Tool)
LangChain 에서 제공하는 빌트인 도구 외에도 사용자가 직접 도구를 정의하여 사용할 수 있습니다.

이를 위해서는 langchain.tools 모듈에서 제공하는 tool 데코레이터를 사용하여 함수를 도구로 변환합니다.

### @tool 데코레이터
이 데코레이터는 함수를 도구로 변환하는 기능을 제공합니다. 다양한 옵션을 통해 도구의 동작을 커스터마이즈할 수 있습니다.

**사용 방법**

1. 함수 위에 @tool 데코레이터 적용
2. 필요에 따라 데코레이터 매개변수 설정

이 데코레이터를 사용하면 일반 Python 함수를 강력한 도구로 쉽게 변환할 수 있으며, 자동화된 문서화와 유연한 인터페이스 생성이 가능합니다.

In [22]:
from langchain.tools import tool

# 데코레이터를 사용하여 함수를 도구로 반환
@tool
def add_number(a:int, b:int) -> int:
    """Add tow number"""
    return a+b

@tool 
def mulply_numbers(a:int, b:int):
    """Multiply two numbers"""
    return a*b

In [25]:
# 도구 실행
add_number.invoke({"a":3, "b": "4"})

7

In [26]:
# 도구 실행
mulply_numbers.invoke({"a":3, "b": "4"})

12

# 구글 뉴스기사 검색 도구
langchain-teddynote 패키지에서 제공하는 GoogleNews 도구를 사용하여 구글 뉴스기사를 검색하는 도구입니다.

**참고**

- API 키가 필요하지 않습니다. (RSS 피드를 사용하기 때문)
news.google.com 에서 제공하는 뉴스기사를 검색하는 도구입니다.

**설명**

- 구글 뉴스 검색 API를 사용하여 최신 뉴스를 검색합니다.
- 키워드를 기반으로 뉴스를 검색할 수 있습니다.
- 최신 뉴스를 검색할 수 있습니다.

**주요 매개변수**

- k (int): 반환할 최대 검색 결과 수 (기본값: 5)
사용하기 전 패키지를 업데이트 해주세요.

In [27]:
#!pip install -qU langchain-teddynote

In [28]:
from langchain_teddynote.tools import GoogleNews
news_tool=GoogleNews()
news_tool.search_latest(k=5)

[{'url': 'https://news.google.com/rss/articles/CBMidEFVX3lxTFBjQ2IzeEc0SnhPcDkyS25qaWpKMWQyMF9SbUtaTnFKLURqZ1g2LTNteURuVlNDS3VJYzJJbFVMSERZZ1A2REwwQXNSYmVSRWxtQ3RyTUhiSmhXaVA2YS1IazBMT0VVZUJpS2VoUUc1U3hWMk9G?oc=5',
  'content': '[단독] 박완수 경남지사 쪽 “2021년 윤 자택 방문…명태균이 제안” - 한겨레'},
 {'url': 'https://news.google.com/rss/articles/CBMiUEFVX3lxTFBiNXhWMzZPNGVoM3FxRDU0dnpXQ1d2OEN3VVIxWENQZkM2YnduallWLXplUUQ2MHJPYmpMQl84SnZrZVBxd3AtUFliRnVDbGFl0gFTQVVfeXFMUFhRZER4WU42cEkwZjFOdGp4ZjRRWC1vdnpoLWN5YWpwVEhsSU1xU0JzMVFHV2Z5VWxqYVdJdTBCN0ZydkNEeS0wMElwZ1A4eUZ2NVU?oc=5',
  'content': '북한 “한국 연결 도로·철도 영구 차단하고 요새화”...‘통일 삭제’ 언급 없이 개헌 - VOA Korea'},
 {'url': 'https://news.google.com/rss/articles/CBMigwFBVV95cUxQQzdYZm9zcHRGa1V1cURBSHRqRFNVQmtoYmluQjlUR00yWUlUY3pFem5IVjVhb1RMNWkwSkIyazdoOS1sQ0RaQmF4d3hvS0EtZzJwV2hJd2ZDcVdGR3lJUDM3NFM4TnduRDhmWVMtQ08tTHBBX0U4OVFhRHk2VnhISmRYa9IBlwFBVV95cUxQZUgyZy1Vam9ZM2VQdG9FN2pDMVFMWUJpd3U4Y3ZsVjRWNzUzYldrbGZlSUVNSTlUdTNQOHlsVWdSblBpUXhqd001T3kteFlvLXhyOTZHNmFCMDdfS2lpaTNq

In [29]:
# 키워드로 뉴스 검색
news_tool.search_by_keyword("AI 투자", k=5)

[{'url': 'https://news.google.com/rss/articles/CBMiWkFVX3lxTE1tdU1haE5HMTRQNXZxNXRYXzBBMDVrZHYyeXEyMzNqc05jaEVPLXlaYUZ2dEVjWEVjdGxfd2lsUTZySWV0bzh2X3dEVXVpR3BXMjN4dFlzUTM2d9IBVEFVX3lxTFBBWDMxcTloY2JnSzVDU1Q1TVJ3aTZkY2lUNFVjOFlDOWVXcF8zcG10eTMtSFkzMk1HazNObURJUFpPS2VCOGhYT0FCM2dndFpUek5OSQ?oc=5',
  'content': '오펜하이머 "MS,오픈AI 투자 손실 위험"…\'중립\'하향 - 한국경제'},
 {'url': 'https://news.google.com/rss/articles/CBMiVkFVX3lxTFBaN3Z4eDhSNTBhamJPQ1hIb0pCeDdBR2N4dmpoQjJYZ1VYMkc0ZEFqN2lPREJNT3AtLTB3c3VHQTBrRHoxZ0JxcnNHcTZmb1JaOUxYZVBR?oc=5',
  'content': '"AI 인프라 구축력 인정"…베슬에이아이, 158억원 투자 유치 - ZD넷 코리아'},
 {'url': 'https://news.google.com/rss/articles/CBMiW0FVX3lxTFBIYnl0dmFrRjBLYWdPRVk1QUtNM2NEQzB3RU1KN3BaZVdmTW42dDlwVlZlajNaSnliR2k0R2NhenRaUGItLVBFdzRXZTVqa21qN2V5cUttbVNKZEHSAV5BVV95cUxNcUVzNmltVl9UX29VU1N6dFVDcW4yU2IzQzMwQ3BsTm5veHM2VGJUUGtkQWw2WEktZzZ2aEJ4NmJtX25kbWt3bklzNlVqNE9RU0VpeC1lbS1Ga1pIZ1h3?oc=5',
  'content': '"엔비디아, 연말 추가 상승 전망…AI 투자 지속 기대" - 연합뉴스'},
 {'url': 'https://news.google.com/rss/ar

In [39]:
from langchain_teddynote.tools import GoogleNews
from langchain.tools import tool
from typing import List, Dict

@tool
def search_keywrod(query: str) -> List[Dict[str, str]]:
    """Look up news by keyword"""
    print(query)
    news_tool = GoogleNews()
    return news_tool.search_by_keyword(query, k=5)

In [40]:
search_keywrod.invoke({"query": "AI투자"})

AI투자


[{'url': 'https://news.google.com/rss/articles/CBMiVkFVX3lxTE9FdmQ4S2VSMmowbU0yYmVMRkxQNjk5Z2EzRGR5LWJFZVZuYVBvLUhoakYwd0tiM2tGczFfZm16TXJCbDYtbjI4eDg5WUlaVWRzeGx1X2F3?oc=5',
  'content': '오펜하이머 "MS,오픈AI 투자 손실 위험"…\'중립\'하향 - 네이트 뉴스'},
 {'url': 'https://news.google.com/rss/articles/CBMiVkFVX3lxTFBaN3Z4eDhSNTBhamJPQ1hIb0pCeDdBR2N4dmpoQjJYZ1VYMkc0ZEFqN2lPREJNT3AtLTB3c3VHQTBrRHoxZ0JxcnNHcTZmb1JaOUxYZVBR?oc=5',
  'content': '"AI 인프라 구축력 인정"…베슬에이아이, 158억원 투자 유치 - ZD넷 코리아'},
 {'url': 'https://news.google.com/rss/articles/CBMiW0FVX3lxTFBIYnl0dmFrRjBLYWdPRVk1QUtNM2NEQzB3RU1KN3BaZVdmTW42dDlwVlZlajNaSnliR2k0R2NhenRaUGItLVBFdzRXZTVqa21qN2V5cUttbVNKZEHSAV5BVV95cUxNcUVzNmltVl9UX29VU1N6dFVDcW4yU2IzQzMwQ3BsTm5veHM2VGJUUGtkQWw2WEktZzZ2aEJ4NmJtX25kbWt3bklzNlVqNE9RU0VpeC1lbS1Ga1pIZ1h3?oc=5',
  'content': '"엔비디아, 연말 추가 상승 전망…AI 투자 지속 기대" - 연합뉴스'},
 {'url': 'https://news.google.com/rss/articles/CBMiTkFVX3lxTE5mMVQ1aGpDQlRDRGh1UkU2VnhhN0lJMl9tN0Q3VTRMZ3RfdFlOekZrelVBTUlvRzFBRHBfYlhObkpKSVF4dTlHUVc3alpLZw?oc

In [41]:
search_keywrod.invoke({"query": "투자가치 있는 주식"})

투자가치 있는 주식


[{'url': 'https://news.google.com/rss/articles/CBMickFVX3lxTE9iM1pxaWNScHdUWXcyNFBFSzU3a2wxZlRFNlVEWENkOFFoSDctdHVkbU10MHlNZVowc2YwaVVuQ05RV0RIYnNwYkhJTzhLYlExN1N0Ti1xVF82SExCZzl1bjdLQ0Etanc0dlJORnh1Zk5MQQ?oc=5',
  'content': '미 증시 고공행진에도 현금 계속 쌓는 기업인들...왜? - 오피니언뉴스'},
 {'url': 'https://news.google.com/rss/articles/CBMiUkFVX3lxTE96S0hUejFMTG5rODN5bndqY2J3QzlreTJOaC1xSUV1SFdLU3A2cUhkbmdfR3F0RkdtaGVTVXdyTGp5ZG12NGotSU9sNHRyVmNyM1E?oc=5',
  'content': '中의 쩐해전술 부양책 … 해외주식 간접투자로 즐긴다 - 매일경제'},
 {'url': 'https://news.google.com/rss/articles/CBMiW0FVX3lxTE1VV3Y5LWNCcWJEcWI4a0hJMkMxcGk2Z0JCbEZNQ0NmYlhlTGRPQ19TVnZXemcwWHBHanBUZVZiWHc3Uksyd3Jfa2lOUlo3VUV3cFMzYmRJTTgtSXM?oc=5',
  'content': '지배구조가 밸류업과 무슨 상관이냐는 대한상의 - 에너지경제'},
 {'url': 'https://news.google.com/rss/articles/CBMiWkFVX3lxTE4wVUxWQy1WQVl0aUtTeEpyZ3pmdWlDWXcwaHFjT0F5YjJENEl3MV83LVl4SFI5THhTM2NGRFBqeVNOYk5RbGNrYjVITVBUNi1seEc4OTdJeTNnQdIBVEFVX3lxTE9vVEhQeWpaUWJCMkNFSlkxRW1sYmxaRXpsdjRBbkJOLUZFTWhGcnBfejFxSU5RRjc5MVBYOWRiQVZzZU5lLUw3VWlT

In [35]:
from langchain.agents import AgentType, initialize_agent
from langchain_community.tools.yahoo_finance_news import YahooFinanceNewsTool
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(temperature=0.0)
tools = [YahooFinanceNewsTool()]
agent_chain = initialize_agent(
    tools,
    llm,
    agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
    verbose=True,
)

C:\Users\skyop\AppData\Local\Temp\ipykernel_15588\1209353012.py:7: LangChainDeprecationWarning: The function `initialize_agent` was deprecated in LangChain 0.1.0 and will be removed in 1.0. Use Use new agent constructor methods like create_react_agent, create_json_agent, create_structured_chat_agent, etc. instead.
  agent_chain = initialize_agent(


In [36]:
agent_chain.invoke(
    "인텔 주식은 최근에 어때?"
)



> Entering new AgentExecutor chain...
I should check the latest financial news about Intel to get an idea of how the stock is performing.
Action: yahoo_finance_news
Action Input: INTC
Observation: Intel Corporation (INTC) Navigates Qualcomm Acquisition Rumors and Antitrust Concerns While Securing Multi-Billion Dollar Deal with AWS for Custom Xeon Chips
We recently compiled a list of the 35 AI Superstars According to Goldman Sachs. In this article, we are going to take a look at where Intel Corporation (NASDAQ:INTC) stands against the other AI superstars according to Goldman Sachs. US technology stocks have surged dramatically this year, largely driven by the growing excitement surrounding generative artificial […]
Thought:Based on the news, it seems like Intel is navigating some challenges but also securing significant deals.
Action: yahoo_finance_news
Action Input: INTC
Observation: Intel Corporation (INTC) Navigates Qualcomm Acquisition Rumors and Antitrust Concerns While Securing 

{'input': '인텔 주식은 최근에 어때?',
 'output': 'Based on the recent financial news, Intel seems to be navigating challenges while also securing significant deals, indicating a dynamic position in the market.'}

In [37]:
agent_chain.invoke(
    "인텔 주식에 대한 동향을 알려줘. 그리고 투자할 가치가 있는지 판단해줘. 한국어로 답해줘"
)



> Entering new AgentExecutor chain...
인텔 주식에 대한 최신 정보를 알아야 한다. 이 정보를 토대로 투자할 가치를 판단할 수 있다.
Action: yahoo_finance_news
Action Input: INTC
Observation: Intel Corporation (INTC) Navigates Qualcomm Acquisition Rumors and Antitrust Concerns While Securing Multi-Billion Dollar Deal with AWS for Custom Xeon Chips
We recently compiled a list of the 35 AI Superstars According to Goldman Sachs. In this article, we are going to take a look at where Intel Corporation (NASDAQ:INTC) stands against the other AI superstars according to Goldman Sachs. US technology stocks have surged dramatically this year, largely driven by the growing excitement surrounding generative artificial […]
Thought:인텔 주식은 Qualcomm 인수 소문과 반독점 우려를 겪고 있지만 AWS와의 수십억 달러 거래를 확보하고 있다. 최신 정보를 토대로 투자할 가치를 판단해야 한다.
Action: yahoo_finance_news
Action Input: INTC
Observation: Intel Corporation (INTC) Navigates Qualcomm Acquisition Rumors and Antitrust Concerns While Securing Multi-Billion Dollar Deal with AWS for Custom Xeon Chips
We r

{'input': '인텔 주식에 대한 동향을 알려줘. 그리고 투자할 가치가 있는지 판단해줘. 한국어로 답해줘',
 'output': '인텔 주식은 현재 다양한 요인에 영향을 받고 있으며, 향후 주가 변동을 예측하여 투자할 가치를 판단해야 한다.'}

In [38]:
agent_chain.invoke(
    "오늘 인텔 주식은 어떻게 됐어?"
)



> Entering new AgentExecutor chain...
I should check the latest financial news about Intel to see how their stock is performing today.
Action: yahoo_finance_news
Action Input: INTC
Observation: Intel Corporation (INTC) Navigates Qualcomm Acquisition Rumors and Antitrust Concerns While Securing Multi-Billion Dollar Deal with AWS for Custom Xeon Chips
We recently compiled a list of the 35 AI Superstars According to Goldman Sachs. In this article, we are going to take a look at where Intel Corporation (NASDAQ:INTC) stands against the other AI superstars according to Goldman Sachs. US technology stocks have surged dramatically this year, largely driven by the growing excitement surrounding generative artificial […]
Thought:I should check the latest financial news about Intel to see how their stock is performing today.
Action: yahoo_finance_news
Action Input: INTC
Observation: Intel Corporation (INTC) Navigates Qualcomm Acquisition Rumors and Antitrust Concerns While Securing Multi-Billio

{'input': '오늘 인텔 주식은 어떻게 됐어?',
 'output': "Intel's stock is currently navigating through various challenges and opportunities."}